# Połączenie z bazą danych za pomocą psycopg2 i pandas

In [1]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')

Ile kategorii filmów mamy w wypożyczalni?

In [2]:
df = pd.read_sql('select count(name) from category',con=connection)
df

,count
0,16


Ile kategorii filmów mamy w wypożyczalni?

In [3]:
df = pd.read_sql('select name from category order by name',con=connection)
df

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [4]:
df = pd.read_sql('select release_year from film order by release_year asc limit 1',con=connection)
df

,release_year
0,2006


Znajdź najstarszy i najmłodszy film do wypożyczenia.

In [5]:
df = pd.read_sql('select release_year from film order by release_year desc limit 1',con=connection)
df

,release_year
0,2006


Ile wypożyczeń odbyło się między 2005-07-01 a 2005-08-01?

In [6]:
df = pd.read_sql("select count(rental_date) from rental where rental_date between '2005-07-01 00:00:00' and '2005-08-01 23:59:59'",con=connection)
df

,count
0,7380


Ile wypożyczeń odbyło się między 2010-01-01 a 2011-02-01?

In [7]:
df = pd.read_sql("select count(rental_date) from rental where rental_date between '2010-01-01 00:00:00' and '2011-02-01 23:59:59'",con=connection)
df

,count
0,0


Znajdź największą płatność wypożyczenia.

In [8]:
df = pd.read_sql('select amount from payment order by amount desc limit 1',con=connection)
df

,amount
0,11.99


Znajdź wszystkich klientów z Polski, Nigerii lub Bangladeszu.

In [9]:
df = pd.read_sql("select first_name, country from customer left join address on customer.address_id = address.address_id left join city on address.city_id = city.city_id left join country on city.country_id = country.country_id where country='Poland' or country='Nigeria' or country='Bangladesh'", con=connection)
df

,first_name,country
0,Sonia,Nigeria
1,Brian,Poland
2,Sidney,Poland
3,Rodney,Nigeria
4,Stephen,Bangladesh
5,Velma,Nigeria
6,Marilyn,Nigeria
7,Elsie,Nigeria
8,Gladys,Nigeria
9,Frank,Bangladesh


Gdzie mieszkają członkowie personelu?

In [10]:
df = pd.read_sql("select address, city, country from staff left join address on staff.address_id = address.address_id left join city on address.city_id = city.city_id left join country on city.country_id = country.country_id", con=connection)
df

,address,city,country
0,23 Workhaven Lane,Lethbridge,Canada
1,1411 Lillydale Drive,Woodridge,Australia


Ilu pracowników mieszka w Argentynie lub Hiszpanii?

In [11]:
df = pd.read_sql("select count(country) from staff left join address on staff.address_id = address.address_id left join city on address.city_id = city.city_id left join country on city.country_id = country.country_id where country='Argentyna' or country = 'Hiszpania'", con=connection)
df

,count
0,0


Jakie kategorie filmów zostały wypożyczone przez klientów?

In [12]:
df = pd.read_sql("select distinct name from category left join film_category on category.category_id = film_category.category_id left join film on film_category.film_id = film.film_id left join inventory on film.film_id = inventory.film_id left join rental on inventory.inventory_id = rental.inventory_id", con=connection)
df

,name
0,Horror
1,Comedy
2,Drama
3,Sci-Fi
4,Foreign
5,Classics
6,Games
7,Travel
8,New
9,Music


Znajdź wszystkie kategorie filmów wypożyczonych w Ameryce.

In [13]:
df = pd.read_sql("select distinct name from category left join film_category on category.category_id = film_category.category_id left join film on film_category.film_id = film.film_id left join inventory on film.film_id = inventory.film_id left join rental on inventory.inventory_id = rental.inventory_id left join customer on rental.customer_id = customer.customer_id left join address on customer.address_id = address.address_id left join city on address.city_id = city.city_id left join country on city.country_id = country.country_id where country = 'United States'", con=connection)
df

,name
0,Action
1,Animation
2,Children
3,Classics
4,Comedy
5,Documentary
6,Drama
7,Family
8,Foreign
9,Games


Znajdź wszystkie tytuły filmów, w których grał: Olympia Pfeiffer lub Julia Zellweger lub Ellen Presley

In [14]:
df = pd.read_sql("select distinct title, first_name, last_name from film left join film_actor on film.film_id = film_actor.film_id left join actor on film_actor.actor_id = actor.actor_id where (first_name || ' '|| last_name) = 'Olympia Pfeiffer' or (first_name || ' '|| last_name) ='Julia Zellweger' or (first_name || ' '|| last_name) = 'Ellen Presley' order by title", con=connection)
df

,title,first_name,last_name
0,Badman Dawn,Olympia,Pfeiffer
1,Bilko Anonymous,Ellen,Presley
2,Breakfast Goldfinger,Julia,Zellweger
3,Caribbean Liberty,Ellen,Presley
4,Casper Dragonfly,Ellen,Presley
...,...,...,...
64,Turn Star,Ellen,Presley
65,Wait Cider,Olympia,Pfeiffer
66,Women Dorado,Ellen,Presley
67,Won Dares,Julia,Zellweger
